# Week 2 課堂練習：爬蟲實戰

## - 用 Python 爬取 Youtube 影片資訊

參考程式碼：[HiSKIO - 用Python爬取 Youtube 資訊 - 圖文課程](https://hiskio.com/courses/112/lectures/3159)

本周練習用 Python 爬取網站資訊，我參考 [HiSKIO | 學程式的好平台](https://hiskio.com/)上 Kevin He 老師的課程，練習用 Python 套件 `Beautifulsoup4` 爬取美國當紅 Youtuber Anna Akana (我的偶像!!!) 的影片資訊，包含影片標題、連結、上傳時間、觀看人數及影片縮圖。

In [33]:
import requests
from bs4 import BeautifulSoup

In [34]:
url = "https://www.youtube.com/results?search_query=anna+akana"
res = requests.get(url)

In [35]:
# Check whether the server successfully answered the http request
res.status_code

200

In [36]:
content = res.content
soup = BeautifulSoup(content, "html.parser")

### ● 網路爬蟲

In [103]:
for video in soup.select(".yt-lockup-video"):
    data = video.select("a[rel='spf-prefetch']")
    # Extract title and link
    print("標題: {}".format(data[0].get("title")))
    print("連結: https://www.youtube.com{}".format(data[0].get("href")))

    # Extract upload time and number of views
    data = video.select(".yt-lockup-meta-info")
    text = data[0].get_text("#").split("#")
    print("上傳時間: {}".format(text[0]))
    if len(text) == 2:
        print("{} 人次".format(text[1]))
    else:
        print("觀看人數: 不明")
        
    # Extract image
    data = video.select("img")
    if data[0].get("src") != "/yts/img/pixel-vfl3z5WfW.gif":
        print("影片縮圖: {}".format(data[0].get("src")))
    else:
        print("影片縮圖: {}".format(data[0].get("data-thumb")))
    print("------------------------------------------------------------------")

標題: Youth & Consequences (Ep 8) - The Fall From Grace
連結: https://www.youtube.com/watch?v=BLnk9xPHcX4
上傳時間: 20 小時前
觀看人數: 不明
影片縮圖: https://i.ytimg.com/vi/BLnk9xPHcX4/hqdefault.jpg?sqp=-oaymwEXCPYBEIoBSFryq4qpAwkIARUAAIhCGAE=&rs=AOn4CLD8AT0x7NywTsQCYJ5h7q6-t-D2jA
------------------------------------------------------------------
標題: Youth & Consequences (Ep 7) - Crotch Riot
連結: https://www.youtube.com/watch?v=nXRzpADzGUU
上傳時間: 20 小時前
觀看人數: 不明
影片縮圖: https://i.ytimg.com/vi/nXRzpADzGUU/hqdefault.jpg?sqp=-oaymwEXCPYBEIoBSFryq4qpAwkIARUAAIhCGAE=&rs=AOn4CLCoC1_2G4NG8EipiBmJq-kS75JM-g
------------------------------------------------------------------
標題: Youth & Consequences (Ep 6) - Tiger Strong
連結: https://www.youtube.com/watch?v=xUIbpXle2Js
上傳時間: 20 小時前
觀看人數: 不明
影片縮圖: https://i.ytimg.com/vi/xUIbpXle2Js/hqdefault.jpg?sqp=-oaymwEXCPYBEIoBSFryq4qpAwkIARUAAIhCGAE=&rs=AOn4CLDgFaXfZDgg9jMvxIgOo6Z-Z_X6Aw
------------------------------------------------------------------
標題: Youth & Consequences (Ep

### ● 網路爬蟲與資料儲存 (Pandas Dataframe)

In [111]:
# Lists to store the scraped data in
titles = []
links = []
times = []
views = []
imgs = []

for video in soup.select(".yt-lockup-video"):
    # Extract titles and links
    data = video.select("a[rel='spf-prefetch']")
    ## title
    title = data[0].get("title")
    titles.append(title)
    
    ## link 
    link = data[0].get("href")
    links.append(link)

    # Extract upload times and number of views
    data = video.select(".yt-lockup-meta-info")
    text = data[0].get_text("#").split("#")
    ## time
    time = text[0]
    times.append(time)
    
    ## views
    if len(text) == 2:
        view = text[1].split("：")[1] # Extract only the numbers
    else:
        view = "不明"
    views.append(view)
        
    # Extract image
    data = video.select("img")
    if data[0].get("src") != "/yts/img/pixel-vfl3z5WfW.gif":
        img = data[0].get("src")
    else:
        img = data[0].get("data-thumb")
    imgs.append(img)

In [113]:
# Store in Pandas Dataframe
test_df = pd.DataFrame({'標題': titles,
                       '連結': links,
                       '上傳時間': times,
                       '觀看人數': views,
                       '影片縮圖': imgs})
test_df = test_df[['標題','連結','上傳時間','觀看人數','影片縮圖']] # Set order of columns

print(test_df.info())
test_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 5 columns):
標題      29 non-null object
連結      29 non-null object
上傳時間    29 non-null object
觀看人數    29 non-null object
影片縮圖    29 non-null object
dtypes: object(5)
memory usage: 1.2+ KB
None


,標題,連結,上傳時間,觀看人數,影片縮圖
0,Youth & Consequences (Ep 8) - The Fall From Grace,/watch?v=BLnk9xPHcX4,20 小時前,不明,https://i.ytimg.com/vi/BLnk9xPHcX4/hqdefault.j...
1,Youth & Consequences (Ep 7) - Crotch Riot,/watch?v=nXRzpADzGUU,20 小時前,不明,https://i.ytimg.com/vi/nXRzpADzGUU/hqdefault.j...
2,Youth & Consequences (Ep 6) - Tiger Strong,/watch?v=xUIbpXle2Js,20 小時前,不明,https://i.ytimg.com/vi/xUIbpXle2Js/hqdefault.j...
3,Youth & Consequences (Ep 5) - Narc-ish,/watch?v=uwsNxbLtIoU,20 小時前,不明,https://i.ytimg.com/vi/uwsNxbLtIoU/hqdefault.j...
4,Youth & Consequences (Ep 4) - Wednesday Night ...,/watch?v=Zr9I9WZS2e0,20 小時前,不明,https://i.ytimg.com/vi/Zr9I9WZS2e0/hqdefault.j...
5,Youth & Consequences (Ep 3) - Gender Fluidity,/watch?v=xhl-kMHFmy0,20 小時前,不明,https://i.ytimg.com/vi/xhl-kMHFmy0/hqdefault.j...
6,Youth & Consequences (Ep 2) - The Hanging Chad...,/watch?v=KjSSU1trCug,20 小時前,不明,https://i.ytimg.com/vi/KjSSU1trCug/hqdefault.j...
7,Youth & Consequences (Ep 1) - The Hanging Chad...,/watch?v=h8ZA40pwpMA,20 小時前,"115,950",https://i.ytimg.com/vi/h8ZA40pwpMA/hqdefault.j...
8,One thing that makes you a better friend,/watch?v=9bAiXJoNdy0,1 週前,"359,213",https://i.ytimg.com/vi/9bAiXJoNdy0/hqdefault.j...
9,What self care really means,/watch?v=zc7G6MJISaI,1 週前,"323,185",https://i.ytimg.com/vi/zc7G6MJISaI/hqdefault.j...
